# Get Tweets in Real time 

In [4]:
pip install tweepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
#import packages
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json

In [2]:
#set the API keys
API_Key = "VXBoAZYLUvGWVWlyBHOog5IN1"
API_Secret_Key = "slkxkVfG5Z9Bm84UPPpTKUnA6IzmAkSMjzHXZ7XSKwx4c26x4f"
Access_Token = "1046485624834969600-CNlee6X66mc4PQnaXDk2Zrpuvpkb8N"
Access_Secret = "gYT41IxSShAAqlC4lWAWaXYUoOp6LLEaELRNjsqqiTWR8"

In [3]:
class TweetsListener(StreamListener): #create a class which uses the StreamListener package
    #build the constructor to initialize the socket
    def __init__(self,csocket): 
        self.client_socket = csocket
    #get the data in json format from the actual tweets and send it to the socket
    def on_data(self,data):
        try: #error handling
            msg = json.loads(data)
            print(msg['text'].encode('utf-8'))
            self.client_socket.send(msg['text'].encode('utf-8'))
            return True
        except BaseException as e:
            print("Error: %s" % str(e))
        return True 

In [4]:
def send_data(API_socket): #define a function which uses the Access keys to extract the actual data
    auth = OAuthHandler(API_Key,API_Secret_Key)
    auth.set_access_token = (Access_Token, Access_Secret)
    stream = Stream(auth, TweetsListener(c_socket)) #passes the c_socket into the TweetsListener Class
    stream.filter(track = ['vaccine', 'vaccination']) #filter the tweets and gets the ones about vaccine

In [5]:
socket.gethostname()

'ip-10-192-11-72'

In [6]:
socket.gethostbyname('ip-10-192-11-72')

'10.192.11.72'

In [7]:
s = socket.socket() #create a socket
host = "10.192.11.72" #local machine name
port = 8087  
s.bind((host,port))
print('Listening on port:  %s' % str(port))

Listening on port:  8087


In [ ]:
s.listen()
c, addr = s.accept() #establish connection with API to listen and process the data
print("Received from :" + str(addr))

In [ ]:
sendData(c) #send the data in real time to the socket be to be used in the next analysis 

# Spark Streaming Analysis for Tweets on Real time

In [ ]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 212.3 MB 23 kB/s s eta 0:00:01     |█████████████████▋              | 117.2 MB 93.3 MB/s eta 0:00:02
     |████████████████████████████████| 198 kB 90.3 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=1c038356a7cadd8b21eb1832a53deef2b098e68cc8eb05b003dfe477ddffb423
  Stored in directory: /home/emr-notebook/.cache/pip/wheels/43/47/42/bc413c760cf9d3f7b46ab7cd6590e8c47ebfd19a7386cd4a57
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [ ]:
sc = SparkContext.getOrCreate()

In [ ]:
ssc = StreamingContext(sc,10) #batch length is 10 sec 

In [ ]:
socket_stream = ssc.socketTextStream("127.0.0.1", 8000) #create the socket stream

In [ ]:
lines = socket_stream.window(30) #window length, tweets tahat are stroed for 30 sec will be analyzed 


In [ ]:
#process the tweets: apply transformation, filter with hashtag symobol (#), convers words to lower case, then use reducebykey to count occurances 
hashtags = lines.flatMap( lambda x: x.split(" ").filter (lambda x: x.lower().startswith("#")).map(lambda x: 
                    (x.lower()),1)).reduceByKey(lambda a,b: a+b)


In [ ]:
#sort the hashtags 
author_counts_sorted = hashtags.transform( lambda foo:foo.sortBy(lambda y:y[0].lower()).sortBy(lambda y:y[1], ascending=False))

In [ ]:
#print the final analysis to extract the popular tweets 
author_counts_sorted.pprint()

In [ ]:
#start the spark streaming context
ssc.start()

In [ ]:
#stop the streaming job 
ssc.awaitTermination()